In [ ]:
#Run this first

from pymatgen import electronic_structure #Make sure that pymatgen is installed
from pymatgen.electronic_structure import plotter
from pymatgen.electronic_structure.plotter import BSDOSPlotter, BSPlotter, DosPlotter
from mp_api.client import MPRester

My_API = 'MJUC3rwGsmc3tEkuE5Q5gOkT3prWnpad' #Find this API Key by going to materialsproject.org -> API -> API Key. (You need an account to do this)

Mat_ID = "mp-21276" #Every material in materialsproject.org has a unique ID attribute to it. Enter this ID here

with MPRester(api_key=My_API) as mpr:
    bs = mpr.get_bandstructure_by_material_id(Mat_ID) #Bandstructure data
    dos = mpr.get_dos_by_material_id(Mat_ID) #Density of states (DOS) data

In [ ]:
#Run this second

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import numpy as np

hw = 22.9 #Photon energy
wf = 4.7 #Work function


#Plot bandstructure
Proj = plotter.BSPlotterProjected(bs)
ax = Proj.get_elt_projected_plots_color() #See the electronic_structure module in pymatgen for more details
ax.ylim([-(hw-wf),0])
ax.xticks(rotation=270)
low, high = ax.xlim()

#Make the bandstructure plot legend
Elm = bs.structure.elements #List of elements in the material
leg = ax.legend(bs.structure.elements,fontsize = 25)
if len(Elm) == 2:
    leg.legend_handles[0].set_color('red')
    leg.legend_handles[1].set_color('blue')
elif len(Elm) == 3:
    leg.legend_handles[0].set_color('red')
    leg.legend_handles[1].set_color('green')
    leg.legend_handles[2].set_color('blue')

ax.ylabel('$E-E_{F}$ (eV)')
ax.xlabel('Wave vector')

#The section below adds green and orange regions to show where MERCEPS overlaps with ARPES
###
#Make the MERCEPS region green (note that doubles can happen outside of this region, but the photoelectrons must have different energies)
Ax = plt.gca()
Xs = [low,high,high,low]
Ys = [-(hw-2*wf)/2,-(hw-2*wf)/2,0,0]
coords = np.zeros([len(Xs),2])
polygons = []
for o in range(len(Xs)):
    for u in range(2):
        if u == 0:
            coords[o][u] = Xs[o]
        else:
            coords[o][u] = Ys[o]
polygons.append(Polygon(coords))
P = PatchCollection(polygons,color='green',alpha = 0.2)
Ax.add_collection(P)

#Make the phantom ARPES region orange
Xs = [low,high,high,low]
Ys = [-(hw-wf),-(hw-wf),-hw/2,-hw/2]
coords = np.zeros([len(Xs),2])
polygons = []
for o in range(len(Xs)):
    for u in range(2):
        if u == 0:
            coords[o][u] = Xs[o]
        else:
            coords[o][u] = Ys[o]
polygons.append(Polygon(coords))
P = PatchCollection(polygons,color='orange',alpha = 0.2)
Ax.add_collection(P)
ax.show()
###


#Plot DOS
dosplot = DosPlotter(sigma=0.1)
dosplot.add_dos("Total DOS", dos)
dosplot.add_dos_dict(dos.get_element_dos())
plt = dosplot.get_plot()
low, high = ax.ylim()
#The section below adds green and orange regions to show where MERCEPS overlaps with ARPES
###
Ax = plt.gca()
Ys = [low,high,high,low]
Xs = [-(hw-2*wf)/2,-(hw-2*wf)/2,0,0]
coords = np.zeros([len(Xs),2])
polygons = []
for o in range(len(Xs)):
    for u in range(2):
        if u == 0:
            coords[o][u] = Xs[o]
        else:
            coords[o][u] = Ys[o]
polygons.append(Polygon(coords))
P = PatchCollection(polygons,color='green',alpha = 0.2)
Ax.add_collection(P)
Ys = [low,high,high,low]
Xs = [-(hw-wf),-(hw-wf),-hw/2,-hw/2]
coords = np.zeros([len(Xs),2])
polygons = []
for o in range(len(Xs)):
    for u in range(2):
        if u == 0:
            coords[o][u] = Xs[o]
        else:
            coords[o][u] = Ys[o]
polygons.append(Polygon(coords))
P = PatchCollection(polygons,color='orange',alpha = 0.2)
Ax.add_collection(P)
###
plt.xlim([-(hw-wf),0])
plt.yticks(rotation=270)
plt.xticks(rotation=270)
plt.xlabel('')
plt.ylabel('')

#The commented code below is meant to change the colors of different elements to be consistent with the BS plots, but it doesnt seem possible without changing the source code.

"""
Elm = bs.structure.elements
if len(Elm) == 2:
    #leg.legend_handles[0].set_color('red')
    #leg.legend_handles[1].set_color('blue')
    #leg.legend_handles[2].set_color('black')
    Ax.plot((), (), "r-",label=Elm[0])
    Ax.plot((), (), "b-",label=Elm[1])
    Ax.plot((), (), "k-",label='Total DOS')
elif len(Elm) == 3:
    #leg.legend_handles[0].set_color('red')
    #leg.legend_handles[1].set_color('green')
    #leg.legend_handles[2].set_color('blue')
    #leg.legend_handles[2].set_color('black')
    Ax.plot((), (), "r-",label=Elm[0])
    Ax.plot((), (), "g-",label=Elm[1])
    Ax.plot((), (), "b-",label=Elm[2])
    Ax.plot((), (), "k-",label='Total DOS')
plt.legend(fontsize=25)
"""
plt.show()

